In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import os


In [2]:
def read_top_instance(file_path):
    with open(file_path, 'r') as file:
        # Read the first three lines
        n_line = file.readline().strip()
        m_line = file.readline().strip()
        tmax_line = file.readline().strip()

        # Parse n N
        _, N = n_line.split()
        N = int(N)

        # Parse m P
        _, P = m_line.split()
        P = int(P)

        # Parse tmax Tmax
        _, Tmax = tmax_line.split()
        Tmax = float(Tmax)

        # Read the remaining lines for each node
        nodes = []
        for i in range(N):
            line = file.readline().strip()
            if line == '':
                continue  # Skip empty lines
            x_str, y_str, s_str = line.split()
            x = float(x_str)
            y = float(y_str)
            s = float(s_str)
            nodes.append({'id': i, 'x': x, 'y': y, 'score': s})
        
        return N, P, Tmax, nodes


In [3]:

def compute_distance_matrix(nodes):
    N = len(nodes)
    distance_matrix = np.zeros((N, N))
    for i in range(N):
        xi, yi = nodes[i]['x'], nodes[i]['y']
        for j in range(N):
            xj, yj = nodes[j]['x'], nodes[j]['y']
            distance = np.hypot(xi - xj, yi - yj)
            distance_matrix[i][j] = distance
    return distance_matrix


In [4]:
def process_all_instances(directory_path):
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            print(f"\nProcessing instance: {filename}")
            N, P, Tmax, nodes = read_top_instance(file_path)
            distance_matrix = compute_distance_matrix(nodes)
            #solve_top_gurobi(N, P, Tmax, nodes, distance_matrix)

# Define the path to your instance directory
instance_directory = 'project/Set_100_234'


In [7]:
def solve_top_gurobi(N, P, Tmax, nodes, distance_matrix):
    try:
        # Create a new model
        model = gp.Model('TOP')

        # Sets
        V = range(N)  # Set of nodes
        K = range(P)  # Set of vehicles

        # Parameters
        c = distance_matrix
        p = [node['score'] for node in nodes]

        # Decision variables
        x = model.addVars(V, V, K, vtype=GRB.BINARY, name='x')
        y = model.addVars(V, K, vtype=GRB.BINARY, name='y')

        # Objective function
        model.setObjective(
            gp.quicksum(p[j] * y[j, k] for j in V for k in K),
            GRB.MAXIMIZE
        )

        # Constraints
        for k in K:
            # Time budget constraint
            model.addConstr(
                gp.quicksum(c[i][j] * x[i, j, k] for i in V for j in V if i != j) <= Tmax,
                name=f'TimeBudget_{k}'
            )

            # Flow conservation
            for j in V:
                if j != 0 and j != N - 1:
                    model.addConstr(
                        gp.quicksum(x[i, j, k] for i in V if i != j) == y[j, k],
                        name=f'FlowIn_{j}_{k}'
                    )
                    model.addConstr(
                        gp.quicksum(x[j, i, k] for i in V if i != j) == y[j, k],
                        name=f'FlowOut_{j}_{k}'
                    )

            # Start at start node
            model.addConstr(
                gp.quicksum(x[0, j, k] for j in V if j != 0) == 1,
                name=f'StartAtStartNode_{k}'
            )

            # End at end node
            model.addConstr(
                gp.quicksum(x[j, N - 1, k] for j in V if j != N - 1) == 1,
                name=f'EndAtEndNode_{k}'
            )

            # No flow into start node and no flow out of end node
            model.addConstr(
                gp.quicksum(x[j, 0, k] for j in V if j != 0) == 0,
                name=f'NoFlowIntoStartNode_{k}'
            )
            model.addConstr(
                gp.quicksum(x[N - 1, j, k] for j in V if j != N - 1) == 0,
                name=f'NoFlowOutOfEndNode_{k}'
            )

        # Each node is visited at most once
        for j in V:
            if j != 0 and j != N - 1:
                model.addConstr(
                    gp.quicksum(y[j, k] for k in K) <= 1,
                    name=f'VisitOnce_{j}'
                )

        # Find the maximum distance
        max_distance = np.max(distance_matrix)

        # Set the big M value
        M = Tmax + max_distance # Or consider using M = Tmax + max_distance if appropriate

        # Subtour elimination constraints (MTZ)
        u = model.addVars(V, K, vtype=GRB.CONTINUOUS, lb=0, ub=Tmax, name='u')

        for k in K:
            for i in V:
                for j in V:
                    if i != j :
                        model.addConstr(
                            u[i, k] + c[i][j] - M * (1 - x[i, j, k]) <= u[j, k],
                            name=f'Subtour_{i}_{j}_{k}'
                        )

        # Optimize the model
        model.optimize()

        # Initialize the routes dictionary
        routes = {}

        # Extract the solution
        if model.Status == GRB.OPTIMAL or model.Status == GRB.TIME_LIMIT:
            for k in K:
                # Get the arcs used by vehicle k
                arcs = [(i, j) for i in V for j in V if x[i, j, k].X > 0.5]
                if not arcs:
                    print(f"Vehicle {k} has no route.")
                    continue
                # Store the arcs in the routes dictionary
                routes[k] = arcs
                # Optionally, print the route
                print(f"\nVehicle {k} route arcs: {arcs}")

                # Reconstruct the route for display purposes (optional)
                arc_dict = {i: j for (i, j) in arcs}
                route = [0]
                current_node = 0
                while current_node != N - 1:
                    if current_node in arc_dict:
                        next_node = arc_dict[current_node]
                        route.append(next_node)
                        current_node = next_node
                    else:
                        print(f"Incomplete route for vehicle {k}.")
                        break
                print(f"Vehicle {k} route nodes: {route}")
            return routes  # Return the dictionary of routes

        elif model.Status == GRB.INFEASIBLE:
            print("Model is infeasible. Computing IIS...")
            # Compute the IIS
            model.computeIIS()
            # Write the IIS to a file
            model.write("model.ilp")
            print("IIS written to 'model.ilp'.")
            # Optionally, write the full model to a file for inspection
            model.write("model.lp")
            print("Model written to 'model.lp'.")
            return None
        else:
            print(f"Optimization was stopped with status {model.Status}")
            return None

    except gp.GurobiError as e:
        print(f"Gurobi Error: {e}")
        return None
    except AttributeError as e:
        print(f"Attribute error: {e}")
        return None


In [8]:
#test with first instance
if __name__ == "__main__":
    # Path to a single instance file
    instance_file = 'Set_100_234/p4.2.a.txt'  # Replace with actual file name

    # Read the instance data
    N, P, Tmax, nodes = read_top_instance(instance_file)

    # Compute the distance matrix
    distance_matrix = compute_distance_matrix(nodes)

    # Solve the TOP using Gurobi
    routes  = solve_top_gurobi(N, P, Tmax, nodes, distance_matrix)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20300 rows, 20400 columns and 119388 nonzeros
Model fingerprint: 0xca07d6aa
Variable types: 200 continuous, 20200 integer (20200 binary)
Coefficient statistics:
  Matrix range     [2e-01, 6e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 4456 rows and 4658 columns
Presolve time: 1.32s
Presolved: 15844 rows, 15742 columns, 92332 nonzeros
Variable types: 196 continuous, 15546 integer (15546 binary)

Root relaxation: objective 3.736404e+02, 6631 iterations, 1.03 seconds (0.92 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

In [ ]:
if __name__ == "__main__":
    # Set the path to your instance directory
    instance_directory = 'Set_100_234'  # Adjust the path if needed

    # Process all instances in the directory
    process_all_instances(instance_directory)


Processing instance: p4.2.a.txt

Processing instance: p4.2.b.txt

Processing instance: p4.2.c.txt

Processing instance: p4.2.d.txt

Processing instance: p4.2.e.txt

Processing instance: p4.2.f.txt

Processing instance: p4.2.g.txt

Processing instance: p4.2.h.txt

Processing instance: p4.2.i.txt

Processing instance: p4.2.j.txt

Processing instance: p4.2.k.txt

Processing instance: p4.2.l.txt

Processing instance: p4.2.m.txt

Processing instance: p4.2.n.txt

Processing instance: p4.2.o.txt

Processing instance: p4.2.p.txt


KeyboardInterrupt: 

NameError: name 'solve_top_gurobi' is not defined